# Analysis of Knee X-Rays of Patients with and without Osteo-Arthritis

In [ ]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import glob
import skimage
from skimage import data, io, filters


### Accessing the Photos
Here, we'll first use glob to gather all the photo paths using glob. Using the glob module, we can recursively extract filenames from our directory using a search string.


Next we'll use a for loop to create an array containing the image as a set of numerical pixel values.


In [ ]:
filenames = glob.glob('**/*.png', recursive = True)
print(filenames[:5])

In [ ]:
'''croping fucntion crops the images to the section of the gap
of the entire image'''

def crop_img(pic):
    img = io.imread(pic)
    cropped = img[65:(img.shape[0]-15),25:(img.shape[1]-15)]
    return cropped


In [ ]:
type(filenames)

### gapwidth()

The def gap_width(img) is used to define the gap width of interest from the knee image. Center = img.shape[1]//2 is taking the center of the image, dividing in half from the top and bottom. While taking the width = 10 horizontally on both sides from the center. We used gaps = [] to define the gap width in columns. By using np.arrange(), we define the area we want relative to the center and width range. The data = img[:,col,0] defines the column/cropped image. Using the min, max, and range function to find the image size, then with np.where() to define the condition. We use np.sum() to add up all the conditions we inputted and gaps.append() to create a list of all the points within the defined gap width. Then the return np.mean() to return the average of the gap.


In [ ]:
def gap_width(img): # two person job (Leilani and Sophia)
    # for p in pics:
    img = crop_img(img)
    center = img.shape[1]//2
    width = 10
    gaps = []
    for col in np.arange(center-width,center+width):
        data = img[:,col] # is this the right column?
        min = np.min(data)
        max = np.max(data)
        range = max-min
        example = np.where(data < (min+0.15*range), 1, 0)
        gap = np.sum(example)
        gaps.append(gap)

    return np.mean(gaps, out=None).tolist()

# Try it out on a few and see if it makes sense
# figure out how to deal with no gap???


### Transphorm()

In [ ]:
'''
transphorm():
    args: file list
     definition: this function takes a file list of knee x-rays and returns
     an array containing the severity and the width between the femur and the meniscus
'''

def transphorm(file):
    data = []
    for f in file:
        crop_img(f)
        width = gap_width(f)
        #print(width)
        data.append(width)
        #data = np.asarray(data)
    return data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def severity(limit_gap):
    if limit_gap < 5: return 'Severe'
    elif limit_gap < 10: return 'Moderate'
    elif limit_gap < 15: return 'Mild'
    elif limit_gap < 20: return 'Doubtful'
    else: return 'Normal'

# limit_gap is the parameter for the function.
# that parameter is a numeric value, it can be int or float or double.
# evaluates by means of an if structure to validate if the limit_gap is less than the set value
# we return a string with the classification of the limit_gap

# width_knees = np.loadtxt(fname='C:/Users/crist/Desktop/Second Year Fall/biomedical signal processsing/Project/width_knees.txt', skiprows=1)
# We declare a variable to load the data, we use numpy's loadtxt. fname is the name of the file
# skiprows allows you to skip or skip lines to avoid loading invalid data. If you have a header, you skip the header
# setting value 1 in skiprows
# print('width_knees', width_knees)
# width_knees # this should change with the array the team send me as in the previous step Cimone


data = transphorm(filenames)

severity_knees = np.array([[width, severity(width)] for width in data], dtype='object')
# We declare a variable called severity_kness.
# We use list for comprehensiveness. list = [value for value in array].
# we assemble a list containing as elements the width and the value returned by the severity function evaluated in width
# np.array allows you to convert a traditional array into a numpy array.
# dtype is to define the data type of the values of the dimensional array. object allows to have several data types
# in the same dimensional array

# for width in width_knees: # usamos for traditional
# severity_knees.append([width, severity(width)]) # add a new item to the list using the append function.

print('severity knees =', severity_knees)

In [ ]:
mean = np.mean(severity_knees[0:,0])
print('mean =', mean) #average of the severity data

max = np.max(severity_knees[0:,0]) # maximum data in an array
min = np.min(severity_knees[0:,0])
print('max =', max, 'min =', min) # minimum data of an array

std = np.std(severity_knees[0:,0])
print('std =', std) # standard deviation of the data

gap = mean + std
print('gap =', gap) # we obtain the width by the sum of the mean and the standard deviation

In [ ]:
"""
    original data
    places the photos by severity in lists
"""
doubtful = []
severe = []
mild = []
normal = []
for x in filenames:
    if 'Doubtful' in x:
        doubtful.append(x)
    elif "Severe" in x:
        severe.append(x)
    elif 'Mild' in x:
        mild.append(x)
    else:
        normal.append(x)

In [ ]:
'''our algorithm
    places the photos by severity in lists from our code '''

doubtful_alg = []
severe_alg = []
mild_alg = []
normal_alg = []
for x in severity_knees[1:,1:]:
    if 'Doubtful' in x:
        doubtful_alg.append(x)
    elif "Severe" in x:
        severe_alg.append(x)
    elif 'Mild' in x:
        mild_alg.append(x)
    else:
        normal_alg.append(x)

In [ ]:
""" getting the the averages of each stage and taking the percent errors
between orignal data and our code """

org_normal = len(normal)/len(normal + mild + severe + doubtful)
own_normal = len(normal_alg)/len(normal_alg + mild_alg + doubtful_alg + severe_alg)
percent_error_normal = (own_normal-org_normal)/org_normal
print(f"The percent error for normal {percent_error_normal*100}%")

org_mild = len(mild)/len(normal + mild + severe + doubtful)
own_mild = len(mild_alg)/len(normal_alg + mild_alg + doubtful_alg + severe_alg)
percent_error_mild = abs((own_mild-org_mild)/org_mild)
print(f"The percent error for mild {percent_error_mild*100}%")

org_doubtful = len(doubtful)/len(normal + mild + severe + doubtful)
own_doubtful = len(doubtful_alg)/len(normal_alg + mild_alg + doubtful_alg + severe_alg)
percent_error_doubtful = abs((own_doubtful-org_doubtful)/org_doubtful)
print(f"The percent error for doubtful {percent_error_doubtful*100}%")

org_severe = len(severe)/len(normal + mild + severe + doubtful)
own_severe = len(severe_alg)/len(normal_alg + mild_alg + doubtful_alg + severe_alg)
percent_error_severe = abs((own_severe-org_severe)/org_severe)
print(f"The percent error for severe {percent_error_severe*100}%")